<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#第12章-使用FP-growth算法来高效发现频繁项集" data-toc-modified-id="第12章-使用FP-growth算法来高效发现频繁项集-1">第12章 使用FP-growth算法来高效发现频繁项集</a></span><ul class="toc-item"><li><span><a href="#FP树-介绍" data-toc-modified-id="FP树-介绍-1.1">FP树 介绍</a></span></li><li><span><a href="#FP-growth-原理" data-toc-modified-id="FP-growth-原理-1.2">FP-growth 原理</a></span></li></ul></li><li><span><a href="#FP-growth-算法优缺点:" data-toc-modified-id="FP-growth-算法优缺点:-2">FP-growth 算法优缺点:</a></span></li><li><span><a href="#从一棵-FP-树中挖掘频繁项集" data-toc-modified-id="从一棵-FP-树中挖掘频繁项集-3">从一棵 FP 树中挖掘频繁项集</a></span></li><li><span><a href="#示例:从新闻网站点击流中挖掘" data-toc-modified-id="示例:从新闻网站点击流中挖掘-4">示例:从新闻网站点击流中挖掘</a></span></li></ul></div>

## 第12章 使用FP-growth算法来高效发现频繁项集


本章会在上一章讨论话题的基础上进行扩展，将给出一个非常好的频繁项集发现算法。该算法称作FP-growth，它比上一章讨论的Apriori算法要快。它基于Apriori构建，但在完成相同任务时采用了一些不同的技术。这里的任务是将数据集存储在一个特定的称作FP树的结构之后发现频繁项集或者频繁项对，即常在一块出现的元素项的集合FP树。

FP-growth算法只需要对数据库进行两次扫描，而Apriori算法对于每个潜在的频繁项集都会扫描数据集判定给定模式是否频繁，因此FP-growth算法的速度要比Apriori算法快。 
它发现频繁项集的基本过程如下： 
(1) 构建FP树 
(2) 从FP树中挖掘频繁项集 

### FP树 介绍
FP树的节点结构如下:
```
class treeNode:
    def __init__(self, nameValue, numOccur, parentNode):
        self.name = nameValue     # 节点名称
        self.count = numOccur     # 节点出现次数
        self.nodeLink = None      # 不同项集的相同项通过nodeLink连接在一起
        # needs to be updated
        self.parent = parentNode  # 指向父节点
        self.children = {}        # 存储叶子节点
```

### FP-growth 原理
基于数据构建FP树

步骤1:

1. 遍历所有的数据集合，计算所有项的支持度。
2. 丢弃非频繁的项。
3. 基于 支持度 降序排序所有的项。

![](./image/Ch12/1.jpg)


1. 所有数据集合按照得到的顺序重新整理。
2. 重新整理完成后，丢弃每个集合末尾非频繁的项。

![](./image/Ch12/2.jpg)


步骤2: 6. 读取每个集合插入FP树中，同时用一个头部链表数据结构维护不同集合的相同项。

![](./image/Ch12/3.jpg)

最终得到下面这样一棵FP树

![](./image/Ch12/4.jpg)

从FP树中挖掘出频繁项集

步骤3:

1. 对头部链表进行降序排序
2. 对头部链表节点从小到大遍历，得到条件模式基，同时获得一个频繁项集。

![](./image/Ch12/5.jpg)

如上图，从头部链表 t 节点开始遍历，t 节点加入到频繁项集。找到以 t 节点为结尾的路径如下:

![](./image/Ch12/6.jpg)

去掉FP树中的t节点，得到条件模式基<左边路径,左边是值>[z,x,y,s,t]:2，[z,x,y,r,t]:1 。条件模式基的值取决于末尾节点 t ，因为 t 的出现次数最小，一个频繁项集的支持度由支持度最小的项决定。所以 t 节点的条件模式基的值可以理解为对于以 t 节点为末尾的前缀路径出现次数。

条件模式基继续构造条件 FP树， 得到频繁项集，和之前的频繁项组合起来，这是一个递归遍历头部链表生成FP树的过程，递归截止条件是生成的FP树的头部链表为空。 根据步骤 2 得到的条件模式基 [z,x,y,s,t]:2，[z,x,y,r,t]:1 作为数据集继续构造出一棵FP树，计算支持度，去除非频繁项，集合按照支持度降序排序，重复上面构造FP树的步骤。最后得到下面 t-条件FP树 :

![](./image/Ch12/7.jpg)


然后根据 t-条件FP树 的头部链表进行遍历，从 y 开始。得到频繁项集 ty 。然后又得到 y 的条件模式基，构造出 ty的条件FP树，即 ty-条件FP树。继续遍历ty-条件FP树的头部链表，得到频繁项集 tyx，然后又得到频繁项集 tyxz. 然后得到构造tyxz-条件FP树的头部链表是空的，终止遍历。我们得到的频繁项集有 t->ty->tyz->tyzx，这只是一小部分。

条件模式基:头部链表中的某一点的前缀路径组合就是条件模式基，条件模式基的值取决于末尾节点的值。
条件FP树:以条件模式基为数据集构造的FP树叫做条件FP树。


## FP-growth 算法优缺点:

* 优点： 1. 因为 FP-growth 算法只需要对数据集遍历两次，所以速度更快。
        2. FP树将集合按照支持度降序排序，不同路径如果有相同前缀路径共用存储空间，使得数据得到了压缩。
        3. 不需要生成候选集。
        4. 比Apriori更快。
* 缺点： 1. FP-Tree第二次遍历会存储很多中间过程的值，会占用很多内存。
        2. 构建FP-Tree是比较昂贵的。
* 适用数据类型：标称型数据(离散型数据)。

In [20]:
#FP树中节点的类定义
class treeNode:
    def __init__(self, nameValue, numOccur, parentNode):
        self.name = nameValue
        self.count = numOccur
        self.nodeLink = None #nodeLink 变量用于链接相似的元素项
        self.parent = parentNode #指向当前节点的父节点
        self.children = {} #空字典，存放节点的子节点

    def inc(self, numOccur):
        self.count += numOccur

#将树以文本形式显示
    def disp(self, ind=1):
        print ('  ' * ind, self.name, ' ', self.count)
        for child in self.children.values():
            child.disp(ind + 1)

In [21]:
#构建FP-tree
def createTree(dataSet, minSup=1):
    headerTable = {}
    for trans in dataSet:  #第一次遍历：统计各个数据的频繁度
        for item in trans:
            headerTable[item] = headerTable.get(item, 0) + dataSet[trans]
            #用头指针表统计各个类别的出现的次数，计算频繁量：头指针表[类别]=出现次数
    for k in list(headerTable.keys()):  #删除未达到最小频繁度的数据
        if headerTable[k] < minSup:
            del (headerTable[k])
    freqItemSet = set(headerTable.keys())#保存达到要求的数据
    # print ('freqItemSet: ',freqItemSet)
    if len(freqItemSet) == 0:
        return None, None  #若达到要求的数目为0
    for k in headerTable: #遍历头指针表
        headerTable[k] = [headerTable[k], None]  #保存计数值及指向每种类型第一个元素项的指针
    # print ('headerTable: ',headerTable)
    retTree = treeNode('Null Set', 1, None)  #初始化tree
    for tranSet, count in dataSet.items():  # 第二次遍历：
        localD = {}
        for item in tranSet:  # put transaction items in order
            if item in freqItemSet:#只对频繁项集进行排序
                localD[item] = headerTable[item][0]

        #使用排序后的频率项集对树进行填充
        if len(localD) > 0:
            orderedItems = [v[0] for v in sorted(localD.items(), key=lambda p: p[1], reverse=True)]
            updateTree(orderedItems, retTree, headerTable, count)  # populate tree with ordered freq itemset
    return retTree, headerTable  #返回树和头指针表


def updateTree(items, inTree, headerTable, count):
    if items[0] in inTree.children:  # 首先检查是否存在该节点
        inTree.children[items[0]].inc(count)  # 存在则计数增加
    else:  # 不存在则将新建该节点
        inTree.children[items[0]] = treeNode(items[0], count, inTree)#创建一个新节点
        if headerTable[items[0]][1] == None:  # 若原来不存在该类别，更新头指针列表
            headerTable[items[0]][1] = inTree.children[items[0]]#更新指向
        else:#更新指向
            updateHeader(headerTable[items[0]][1], inTree.children[items[0]])
    if len(items) > 1:  #仍有未分配完的树，迭代
        updateTree(items[1::], inTree.children[items[0]], headerTable, count)

#节点链接指向树中该元素项的每一个实例。
# 从头指针表的 nodeLink 开始,一直沿着nodeLink直到到达链表末尾
def updateHeader(nodeToTest, targetNode):
    while (nodeToTest.nodeLink != None):
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode

后面构建树时会 
使用 createTree() 函数,而该函数的输入数据类型不是列表。其需要的是一部字典,其中项集为字典中的键,而频率为每个键对应的取值。 createInitSet() 用于实现上述从列表到字典的类型转换过程。

In [22]:
def loadSimpDat():
    simpDat = [['r', 'z', 'h', 'j', 'p'],
               ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
               ['z'],
               ['r', 'x', 'n', 'o', 's'],
               ['y', 'r', 'x', 'z', 'q', 't', 'p'],
               ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]
    return simpDat
#createInitSet() 用于实现上述从列表到字典的类型转换过程
def createInitSet(dataSet):
    retDict = {}
    for trans in dataSet:
        retDict[frozenset(trans)] = 1
    return retDict

直接参考书中代码运行报错：RuntimeError: dictionary changed size during iteration 
解决：修改createTree()函数的for k in headerTable.keys(): 为for k in list(headerTable): 

In [23]:
simpDat=loadSimpDat()
simpDat

[['r', 'z', 'h', 'j', 'p'],
 ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
 ['z'],
 ['r', 'x', 'n', 'o', 's'],
 ['y', 'r', 'x', 'z', 'q', 't', 'p'],
 ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]

In [24]:
initSet=createInitSet(simpDat)
initSet

{frozenset({'h', 'j', 'p', 'r', 'z'}): 1,
 frozenset({'s', 't', 'u', 'v', 'w', 'x', 'y', 'z'}): 1,
 frozenset({'z'}): 1,
 frozenset({'n', 'o', 'r', 's', 'x'}): 1,
 frozenset({'p', 'q', 'r', 't', 'x', 'y', 'z'}): 1,
 frozenset({'e', 'm', 'q', 's', 't', 'x', 'y', 'z'}): 1}

In [27]:
myFPtree,myHeaderTab=createTree(initSet,3)
myFPtree.disp()

   Null Set   1
     z   5
       r   1
       x   3
         s   2
           t   2
             y   2
         r   1
           t   1
             y   1
     x   1
       s   1
         r   1


上面给出的是元素项及其对应的频率计数值,其中每个缩进表示所处的树的深度。可以验证一下这棵树上图中所示的树是否等价

## 从一棵 FP 树中挖掘频繁项集 
从FP树中抽取频繁项集的三个基本步骤如下:

>(1) 从FP树中获得条件模式基;
(2) 利用条件模式基,构建一个条件FP树;
(3) 迭代重复步骤(1)步骤(2),直到树包含一个元素项为止。

接下来首先寻找条件模式基的过程。之后为每一个条件模式基创建对应的条件FP树。最后需要构造少许代码来封装上述两个函数,并从FP树中获得频繁项集。

**条件模式基是以所查找元素项为结尾的路径集合。**  

每一条路径其实都是一条前缀路径(prefix path)。简而言之,一条前缀路径是介于所查找元素项与树根节点之间的所有内容。







In [28]:
#从FP树中发现频繁项集
def ascendTree(leafNode, prefixPath):  #递归上溯整棵树
    if leafNode.parent != None:
        prefixPath.append(leafNode.name)
        ascendTree(leafNode.parent, prefixPath)


def findPrefixPath(basePat, treeNode):  #参数：指针，节点；
    condPats = {}
    while treeNode != None:
        prefixPath = []
        ascendTree(treeNode, prefixPath)#寻找当前非空节点的前缀
        if len(prefixPath) > 1:
            condPats[frozenset(prefixPath[1:])] = treeNode.count #将条件模式基添加到字典中
        treeNode = treeNode.nodeLink
    return condPats

In [34]:
findPrefixPath('x',myHeaderTab['x'][1])

{frozenset({'z'}): 3}

In [35]:
findPrefixPath('x',myHeaderTab['r'][1])

{frozenset({'z'}): 1, frozenset({'s', 'x'}): 1, frozenset({'x', 'z'}): 1}

对于每一个频繁项都要创建一棵条件FP树，使用条件模式基作为输入数据，用相同的建树代码构建条件树，之后递归地发现频繁项、发现条件模式基，并且继续构造条件树，直到条件树中没有元素。

函数mineTree对参数inTree代表的FP树进行频繁项集挖掘。首先对headerTable中出现的单个元素按出现频率从小到大排序，之后将每个元素的条件模式基作为输入数据，建立针对当前元素的条件树，如果生成的这棵条件树仍有元素，就在这棵条件树里寻找频繁项集，因为prefix参数是在递归过程中不断向下传递的，因此由最初的headerTable中的某个元素x衍生出的所有频繁项集都带有x。

In [40]:
#递归查找频繁项集
def mineTree(inTree, headerTable, minSup, preFix, freqItemList):
    # 头指针表中的元素项按照频繁度排序,从小到大
    bigL = [v[0] for v in sorted(headerTable.items(), key=lambda p: str(p[1]))]#python3修改
    for basePat in bigL:  #从底层开始
        #加入频繁项列表
        newFreqSet = preFix.copy()
        newFreqSet.add(basePat)
#         print ('finalFrequent Item: ',newFreqSet)
        freqItemList.append(newFreqSet)
        #递归调用函数来创建基
        condPattBases = findPrefixPath(basePat, headerTable[basePat][1])
#         print ('condPattBases :',basePat, condPattBases)

        #2. 构建条件模式Tree
        myCondTree, myHead = createTree(condPattBases, minSup)
        #将创建的条件基作为新的数据集添加到fp-tree
#         print ('head from conditional tree: ', myHead)
        if myHead != None: #3. 递归
#             print ('conditional tree for: ',newFreqSet)
            myCondTree.disp(1)
            mineTree(myCondTree, myHead, minSup, newFreqSet, freqItemList)

测试以上代码会报错：TypeError: unorderable types: treeNode() < treeNode()原因是python3：comparing integers and strings is not allowed，因此需要修改函数中的为bigL = [v[0] for v in sorted(headerTable.items(), key=lambda p: str(p[1]))]添加str将类型转化为字符串类型。 
修改参考下列链接说明：https://www.peterbe.com/plog/sorting-mixed-type-lists-in-python-3 
测试结果：显示所有的条件树 

In [41]:
freqItems=[]
mineTree(myFPtree,myHeaderTab,3,set([]),freqItems)

   Null Set   1
     x   3
       z   3
   Null Set   1
     x   3
   Null Set   1
     x   3
   Null Set   1
     t   3
       x   3
         z   3
   Null Set   1
     t   3
   Null Set   1
     t   3
       x   3
   Null Set   1
     t   3
   Null Set   1
     z   3


In [43]:
freqItems

[{'t'},
 {'t', 'x'},
 {'t', 'z'},
 {'t', 'x', 'z'},
 {'s'},
 {'s', 'x'},
 {'r'},
 {'y'},
 {'x', 'y'},
 {'t', 'x', 'y'},
 {'t', 'y'},
 {'y', 'z'},
 {'t', 'y', 'z'},
 {'x', 'y', 'z'},
 {'t', 'x', 'y', 'z'},
 {'x'},
 {'x', 'z'},
 {'z'}]

## 示例:从新闻网站点击流中挖掘 
在上传的源码中,有一个kosarak.dat文件,它包含将近100万条记录 。该文件中的每一行包含某个用户浏览过的新闻报道。一些用户只看过一篇报道,而有些用户看过2498篇报道。 
用户和报道被编码成整数,所以查看频繁项集很难得到更多的东西,但是该数据对于展示FP-growth算法的速度十分有效。

首先放python3测试结果：